<a href="https://colab.research.google.com/github/qsquentinsmith/BigDataScience/blob/master/03_apache_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INET 4710: Lab 03
## Author: Quentin Smith
### smit8819

## 0. Installing and Running Spark on Google colab

The following code will 
- Get newest versions of packages and dependencies. 
- Download jdk8 and spark 3.0.1 with hadoop 2.7
- Download findspark (to make pyspark importable as a regular library)
- Import os
- Set paths for Java_Home and Spark_Home
- Import findspark and initiate it

In [99]:
# Get newest versions of packages and dependencies.
!apt-get update

# Download jdk8 and spark 3.0.1 with hadoop 2.7
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://download.java.net/openjdk/jdk8u41/ri/openjdk-8u41-b04-linux-x64-14_jan_2020.tar.gz
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Download findspark (to make pyspark importable as a regular library)
!pip install -q findspark

#Import os
import os

# Set paths for Java_Home and Spark_Home
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

# Import findspark and initiate it
import findspark
findspark.init() 

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

### 0.1 Configuring a Spark Session

The entry point to using Spark SQL is an object called SparkSession. This code will initiate a Spark Application which all the code for that session will run on. 

In [100]:
# Configure Spark session and run session
from pyspark.sql import SparkSession
spark = SparkSession.builder \
	.master("local[*]") \
	.appName("Learning_Spark") \
	.getOrCreate() 

## 1. Loading Data

For this guide we’ll be working with a dataset on video game sales from Kaggle. It can be found here.

https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings


In [101]:
# Reads directly into DataFrame in PySparkSQL


url = "https://raw.githubusercontent.com/qsquentinsmith/BigDataScience/master/data/Video_Games_Sales_as_at_22_Dec_2016.csv"

from pyspark import SparkFiles

spark.sparkContext.addFile(url)

data = spark.read.csv("file://"+SparkFiles.get("Video_Games_Sales_as_at_22_Dec_2016.csv"), header=True, inferSchema= True)



## 2. Data Exploration

In [102]:
# Shape
data.count(), len(data.columns)

(16719, 16)

In [103]:
data.show(5)

+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|                Name|Platform|Year_of_Release|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|
+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|          Wii Sports|     Wii|           2006|      Sports| Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322| Nintendo|     E|
|   Super Mario Bros.|     NES|           1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|        null|        null|      null|      null|     null|  null|
|      Mario Kart Wii|     Wii|           2008|      Racing|

We see a couple things from the first two data exploration cells. The first one gives us an idea of how big the data is and how many columns we are working with. The second shows the types of most of the columns (Sometime you cannot tell the type which the next cell will show). The following cell data.printSchema() tells us that Year_of_Release although a number is actually a string in this data frame.

In [104]:
# Shows column titles and types
data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year_of_Release: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)
 |-- Critic_Score: integer (nullable = true)
 |-- Critic_Count: integer (nullable = true)
 |-- User_Score: string (nullable = true)
 |-- User_Count: integer (nullable = true)
 |-- Developer: string (nullable = true)
 |-- Rating: string (nullable = true)



In [105]:
#Choose individual columns
data.select("Name","Platform","User_Score","User_Count").show(15, truncate=False)

+---------------------------+--------+----------+----------+
|Name                       |Platform|User_Score|User_Count|
+---------------------------+--------+----------+----------+
|Wii Sports                 |Wii     |8         |322       |
|Super Mario Bros.          |NES     |null      |null      |
|Mario Kart Wii             |Wii     |8.3       |709       |
|Wii Sports Resort          |Wii     |8         |192       |
|Pokemon Red/Pokemon Blue   |GB      |null      |null      |
|Tetris                     |GB      |null      |null      |
|New Super Mario Bros.      |DS      |8.5       |431       |
|Wii Play                   |Wii     |6.6       |129       |
|New Super Mario Bros. Wii  |Wii     |8.4       |594       |
|Duck Hunt                  |NES     |null      |null      |
|Nintendogs                 |DS      |null      |null      |
|Mario Kart DS              |DS      |8.6       |464       |
|Pokemon Gold/Pokemon Silver|GB      |null      |null      |
|Wii Fit                

In [106]:
# Take closer look at user score and user count
data.describe(["User_Score","User_Count"]).show()

+-------+------------------+------------------+
|summary|        User_Score|        User_Count|
+-------+------------------+------------------+
|  count|             10015|              7590|
|   mean|7.1250461133070315|162.22990777338603|
| stddev|1.5000060936257986| 561.2823262473789|
|    min|                 0|                 4|
|    max|               tbd|             10665|
+-------+------------------+------------------+



We see another problem we will eventually run into. There are various null values and values listed with tbd. We will filter those out in the next session so we can run our model.

In [107]:
# Look at platform distribution
data.groupBy("Platform").count().orderBy("count", ascending=False).show(10)

+--------+-----+
|Platform|count|
+--------+-----+
|     PS2| 2161|
|      DS| 2152|
|     PS3| 1331|
|     Wii| 1320|
|    X360| 1262|
|     PSP| 1209|
|      PS| 1197|
|      PC|  974|
|      XB|  824|
|     GBA|  822|
+--------+-----+
only showing top 10 rows



### 2.1 Filtering DataFrames

Lets create a new DataFrame that has the null values for User_Score and User_Count, and the “tbd” values filtered out using the .filter() method. This will keep the integrity of the original data frame in 'data' and our cleaned data frame 'data2'.

In [108]:
# Check for nulls
from pyspark.sql.functions import col,isnan,when,count
df2 = data.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in data.columns])
df2.show()

+----+--------+---------------+-----+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|Name|Platform|Year_of_Release|Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|
+----+--------+---------------+-----+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|   6|       0|              0|    2|        1|       0|       0|       0|          0|           0|        8582|        8582|      6704|      9129|     6624|  6769|
+----+--------+---------------+-----+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+



In [109]:
#Create conditions to filter
condition1 = (data.User_Score.isNotNull()) | (data.User_Count.isNotNull())
condition2 = data.User_Score != "tbd"
condition3 = data.Year_of_Release != 'N/A'
condition4 = data.Critic_Score.isNotNull()


#Apply filters. Check data
data = data.filter(condition1).filter(condition2).filter(condition3).filter(condition4)
data.describe(["User_Score","User_Count", "Year_of_Release", "Critic_Score"]).show()

+-------+------------------+------------------+------------------+------------------+
|summary|        User_Score|        User_Count|   Year_of_Release|      Critic_Score|
+-------+------------------+------------------+------------------+------------------+
|  count|              6894|              6894|              6894|              6894|
|   mean| 7.184377719756343|174.39237017696547|2007.4823034522774| 70.25848563968668|
| stddev|1.4398056814955311| 584.8721554479154| 4.236401444992017|13.861082448459156|
|    min|               0.5|                 4|              1985|                13|
|    max|               9.6|             10665|              2016|                98|
+-------+------------------+------------------+------------------+------------------+



In [110]:
# change all predictors to type Double
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

# Make a copy of 'data' and name it 'data2'
data2 = data

# Change variables used in model to type: double, using cast
data2 = data2.withColumn("Year_of_Release", data2["Year_of_Release"].cast(DoubleType()))
data2 = data2.withColumn("User_Score", data2["User_Score"].cast(DoubleType()))
data2 = data2.withColumn("User_Count", data2["User_Count"].cast(DoubleType()))
data2 = data2.withColumn("Critic_Score", data2["Critic_Score"].cast(DoubleType())) 

## 3.1 Building Models in PySpark

For an example of linear regression, let’s see if we can predict User_Score from Year_of_Release, Global_Sales, Critic_Score, and User_Count.



In [111]:
# check to see change implemented
data2.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year_of_Release: double (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)
 |-- Critic_Score: double (nullable = true)
 |-- Critic_Count: integer (nullable = true)
 |-- User_Score: double (nullable = true)
 |-- User_Count: double (nullable = true)
 |-- Developer: string (nullable = true)
 |-- Rating: string (nullable = true)



### 3.1 Vector Assembler

Change data into new data frame that PySpark model can use by using vectorassembler. Vector Assumbly puts all the variables we use to predict the target variable (User_Score) into a vector (predictors). This vector is then stored in one column while User_Score is in the other column. 

In [112]:
# Resource Used for set handle invalid https://stackoverflow.com/questions/41362295/sparkexception-values-to-assemble-cannot-be-null.
# Other resource: https://spark.apache.org/docs/latest/ml-features#vectorassembler

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# define vectorassembler
assembler = VectorAssembler(inputCols=['Year_of_Release', 'Global_Sales', 'Critic_Score', 'User_Count'], outputCol = 'predictors')

# output = assembler.setHandleInvalid("skip").transform(data2)
output = assembler.transform(data2)
print("Assembled columns 'Year_of_Release', 'Global_Sales', 'Critic_Score', 'User_Count' to vector column 'predictors'")

#Collect the new predictions column and User_Score (our target variable) in a DataFrame.
model_data = output.select(["predictors", "User_Score"])

model_data.show(5, truncate = False)

Assembled columns 'Year_of_Release', 'Global_Sales', 'Critic_Score', 'User_Count' to vector column 'predictors'
+-------------------------+----------+
|predictors               |User_Score|
+-------------------------+----------+
|[2006.0,82.53,76.0,322.0]|8.0       |
|[2008.0,35.52,82.0,709.0]|8.3       |
|[2009.0,32.77,80.0,192.0]|8.0       |
|[2006.0,29.8,89.0,431.0] |8.5       |
|[2006.0,28.92,58.0,129.0]|6.6       |
+-------------------------+----------+
only showing top 5 rows



### 3.2 Model Training

We will be using Linear Regression as our model. First we need to split the model_data into test and train splits. We will use 80% of the data from 'data2' for the training group and the other 20% for the test split. 

In [113]:
from pyspark.ml.regression import LinearRegression

#Split data into train and test
train, test = model_data.randomSplit([0.8, 0.2])

#Define Linear Regression model
lr = LinearRegression(
	featuresCol = 'predictors',
	labelCol = 'User_Score')

#Fit the model to the training data
lrModel = lr.fit(train)

#Use evaluate class to see our prediction
pred = lrModel.evaluate(test)

Now we can use seperate tools within our models to see the coefficients and the intercept to get a formula to predict future values. We can also use predictions from the class to see our predictions on the test set. 

In [114]:
#access  parameters of our model: coefficients
lrModel.coefficients

DenseVector([-0.0768, -0.018, 0.063, -0.0002])

In [115]:
#access  parameters of our model: intercept
lrModel.intercept

156.89650865430528

In [116]:
#View final predictions
pred.predictions.show(5) 

+--------------------+----------+------------------+
|          predictors|User_Score|        prediction|
+--------------------+----------+------------------+
|[1988.0,0.03,64.0...|       2.2|7.3952122349159595|
|[1994.0,1.27,69.0...|       6.3| 8.175145120381075|
|[1997.0,0.07,79.0...|       7.8| 8.596646967056756|
|[1997.0,0.89,83.0...|       8.2| 8.830229615949833|
|[1998.0,0.05,69.0...|       7.5| 7.890080746311099|
+--------------------+----------+------------------+
only showing top 5 rows



### 3.3 Model Evaluation

We will use RegressionEvaluator to learn whether or not our linear regression was a good model to use on this data. There will be different metrics that we will use including: 
- Root-Mean-Square Error (RMSE)
- Mean-Square Error (MSE)
- Mean Absolute Error (MEA)
- R-Squared (r2)

In [117]:
from pyspark.ml.evaluation import RegressionEvaluator

#DefineRegression Evaluator, Set Parameters)
eval = RegressionEvaluator(
	labelCol="User_Score",
	predictionCol="prediction",
	metricName="rmse")

In [118]:
#run different metrics
rmse = eval.evaluate(pred.predictions)
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})

#print metrics
print("Root-Mean-Square Error (RMSE): ", rmse)
print("Mean-Square Error (MSE): ", mse)
print("Mean Absolute Error (MAE): ", mae)
print("R-squared (r2): ", r2)

Root-Mean-Square Error (RMSE):  1.147240886673481
Mean-Square Error (MSE):  1.3161616520553547
Mean Absolute Error (MAE):  0.869876244720776
R-squared (r2):  0.3832867984279704


# Conclusion

Our evaluation on our model Linear Regression shows the following metrics: 

In [119]:
#print metrics
print("Root-Mean-Square Error (RMSE): ", rmse)
print("Mean-Square Error (MSE): ", mse)
print("Mean Absolute Error (MAE): ", mae)
print("R-squared (r2): ", r2)

Root-Mean-Square Error (RMSE):  1.147240886673481
Mean-Square Error (MSE):  1.3161616520553547
Mean Absolute Error (MAE):  0.869876244720776
R-squared (r2):  0.3832867984279704


**Root-Mean-Square Error (RMSE)** is the standard deviation of the prediction errors. It tells you how concentrated the data is around the line of best fit. So the prediction errors (residuals) have a standard deviation of 1.15. 

**Mean-Square Error (MSE)** tells you how close a regression line is to a set of points. The smaller the mean squared error, the closer you are to finding the line of best fit. The MSE is 1.32.

**Mean Absolute Error (MAE)** is mean of all the difference between the measured value and 'true' value. So the mean difference between our model predicted values and the true value is 0.87.

**R-Squared (r2)** also called Coefficient of Determination. It is used to analyze how difference in one variable can be explained by a difference in a second variable. The percentage indicates that about 38% of predicted points should fall within the regression. Other things this let's us know is how much of a connection there is between the target variable and the predictors. 

For a first try this is a good start. These perdictor variables do have some connection to the target varaible. The next steps we should take is using a feature selection method so we can determine the best predictor variables. We should also use cross validation to fight the bit of overfitting that might be happening. 


**Resource:**

https://www.statisticshowto.com/probability-and-statistics/regression-analysis/rmse-root-mean-square-error/

https://www.statisticshowto.com/mean-squared-error/

https://www.statisticshowto.com/absolute-error/

https://www.statisticshowto.com/probability-and-statistics/coefficient-of-determination-r-squared/

